In [1]:
# RNN, LSTM, GRU ve diğer katmanları import ediyoruz
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding

# IMDB veri setini import ediyoruz
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
import numpy as np

# Kelime dağarcığı boyutunu belirliyoruz
vocab_size = 5000  # Sadece en sık kullanılan 5000 kelimeyi dikkate alacağız

# IMDB veri setini yükleyip eğitim ve test verisine ayırıyoruz
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Eğitim verisindeki ilk örneği yazdırıyoruz (kelimeler indekslerle temsil ediliyor)
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


Bu kod bloğu ile IMDB film yorumları veri seti yükleniyor ve duygu analizi için hazırlanıyor. Veri setindeki yorumlar, kelimeler yerine her kelimenin indeks değerleri ile temsil ediliyor. Kelime dağarcığı boyutu 5000 olarak belirlenmiş, yani en sık kullanılan 5000 kelime dikkate alınacak; geri kalan kelimeler görmezden gelinecek. x_train ve x_test dizileri, yorumlardaki kelime indekslerini içerirken, y_train ve y_test dizileri her yorumun pozitif (1) veya negatif (0) olduğunu gösteriyor.

In [2]:
# IMDB veri setindeki kelimelerin indekslerini ve kelimeleri alıyoruz
word_idx = imdb.get_word_index()

# Sözlüğün anahtar ve değerlerini ters çeviriyoruz (indeks → kelime)
word_idx = {i: word for word, i in word_idx.items()}

# Eğitim verisindeki ilk yorumun kelimelerini indeksler yerine kelime olarak yazdırıyoruz
print([word_idx[i] for i in x_train[0]])

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

Bu kod bloğu, eğitim verisindeki yorumların hangi kelimelerden oluştuğunu anlamamıza yarıyor. Başlangıçta her kelime sadece bir indeks ile temsil ediliyor. get_word_index() fonksiyonu ile indeks-kelime ilişkisini alıyor ve ardından bu ilişkiyi ters çevirerek indeksleri gerçek kelimelerle eşleştiriyoruz. Böylece x_train içindeki ilk yorum, kelime indeksleri yerine okunabilir kelimeler hâline geliyor

In [3]:
# Eğitim ve test veri setindeki yorumların maksimum ve minimum uzunluklarını yazdırıyoruz
print("Max length of a review:", len(max((x_train + x_test), key=len)))
print("Min length of a review:", len(min((x_train + x_test), key=len)))

Max length of a review: 2697
Min length of a review: 70


Bu kod bloğu, veri setindeki yorumların uzunluklarını inceliyor. max ve min fonksiyonları kullanılarak tüm eğitim ve test yorumları arasında en uzun ve en kısa yorumların kaç kelimeden oluştuğu hesaplanıyor. Bu bilgi, modelin giriş verilerini sabit bir uzunlukta hazırlamak (padding/truncating) için önemlidir. Uzunluk farkları çok fazla ise, kısa yorumlar sıfır ile doldurulabilir veya uzun yorumlar belirli bir sınırda kesilebilir. Böylece RNN/GRU/LSTM modelleri için tutarlı bir giriş boyutu sağlanmış olur.

In [4]:
# Sabit uzunlukta girdi dizileri oluşturmak için Keras sequence modülünü kullanıyoruz
from tensorflow.keras.preprocessing import sequence

# Tüm yorumları maksimum 400 kelimeye kadar kesiyoruz veya eksikse sıfır ile dolduruyoruz
max_words = 400
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

# Eğitim sırasında modelin performansını değerlendirmek için küçük bir doğrulama seti ayırıyoruz
x_valid, y_valid = x_train[:64], y_train[:64]       # İlk 64 örnek doğrulama seti
x_train_, y_train_ = x_train[64:], y_train[64:]     # Kalan örnekler eğitim seti

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Her kelimenin embedding (yerleştirme) boyutu
embd_len = 32  

RNN_model = Sequential(name="Simple_RNN")
RNN_model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embd_len))  # input_length kaldırıldı
RNN_model.add(SimpleRNN(128, activation='tanh'))
RNN_model.add(Dense(1, activation='sigmoid'))

# Modeli derle
RNN_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Modeli build et (girdi boyutu burada belirtiliyor)
RNN_model.build(input_shape=(None, max_words))  

# Özeti yazdır
print(RNN_model.summary())

Model: "Simple_RNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 400, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 180,737 (706.00 KB)

 Trainable params: 180,737 (706.00 KB)

 Non-trainable params: 0 (0.00 B)

None


Bu kod bloğu ile oluşturulan model özetinde, kullanılan katmanların türü, çıkış boyutları ve parametre sayıları görülmektedir. İlk katman olan Embedding, her kelimeyi 32 boyutlu bir vektörle temsil ederek sözcükleri daha anlamlı bir şekilde ifade eder. Ardından gelen SimpleRNN katmanı, 128 nöronla kelime sırasındaki bağımlılıkları yakalar ve metnin duygusunu çıkarmak için zaman boyutundaki bilgileri işler. Son olarak Dense katmanı, sigmoid aktivasyonu sayesinde ikili sınıflandırma yapar ve yorumların pozitif ya da negatif olarak etiketlenmesini sağlar. Toplam 180,737 parametrenin tamamı eğitilebilir olup, bu da modelin öğrenme kapasitesini göstermektedir. Bu yapı sayesinde model, kelimeler arasındaki bağlamı dikkate alarak duygu analizi yapabilecek şekilde hazırlanmıştır.

In [7]:
#Modelin eğitimi
history=RNN_model.fit(x_train_,y_train_,batch_size=64,epochs=5,verbose=1,validation_data=(x_valid,y_valid))
#Sonuçların yazdırılması
print()
print("Simple_RNN Score--->",RNN_model.evaluate(x_test,y_test,verbose=0))

Epoch 1/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 61s 149ms/step - accuracy: 0.5985 - loss: 0.6416 - val_accuracy: 0.7656 - val_loss: 0.5441
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 52s 132ms/step - accuracy: 0.7048 - loss: 0.5631 - val_accuracy: 0.6719 - val_loss: 0.6466
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 44s 113ms/step - accuracy: 0.7808 - loss: 0.4693 - val_accuracy: 0.8438 - val_loss: 0.4316
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 50s 128ms/step - accuracy: 0.7998 - loss: 0.4359 - val_accuracy: 0.8438 - val_loss: 0.4417
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 59s 151ms/step - accuracy: 0.8127 - loss: 0.4201 - val_accuracy: 0.8281 - val_loss: 0.4508

Simple_RNN Score---> [0.4215129315853119, 0.8198400139808655]


Bu kod bloğunda model, IMDb duygu analizi veri seti üzerinde 5 epoch boyunca eğitilmiştir. Eğitim sürecinde doğruluk (accuracy) değerinin her epoch ile birlikte arttığı, kayıp (loss) değerinin ise azaldığı görülmektedir. Bu durum, modelin eğitim verisini giderek daha iyi öğrendiğini göstermektedir.

Doğrulama sonuçlarına bakıldığında, val_accuracy değerinin 3. epoch itibarıyla %84 civarında sabitlendiği, val_loss değerinin ise 3. epoch sonrası küçük dalgalanmalar gösterdiği gözlemlenmektedir. Bu, modelin doğrulama verisi üzerinde makul bir genelleme yapabildiğini ancak aşırı öğrenmeye (overfitting) eğilim gösterebileceğini işaret eder.

Eğitim tamamlandıktan sonra test verisi üzerinde yapılan değerlendirmede %81.9 doğruluk elde edilmiştir. Bu oran, Simple RNN modelinin duygu analizi görevinde başarılı bir performans sergileyebildiğini ortaya koymaktadır.

In [8]:
#Aynı işlem GRU ile tanımlandı
#GRU = tekrarlayan kapı kazancı
gru_model=Sequential(name="GRU_Model")
gru_model.add(Embedding(vocab_size, embd_len, input_length=max_words))
gru_model.add(GRU(128,activation="tanh",return_sequences=False))
gru_model.add(Dense(1,activation="sigmoid"))
gru_model.build(input_shape=(None, max_words))

#Model özetinin yazdırılması
print(gru_model.summary())
#Modelin derlenmesi
gru_model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
#Modelin eğitimi
history2=gru_model.fit(x_train_,y_train_,batch_size=64,epochs=5,verbose=1,validation_data=(x_valid,y_valid))
#Test sonuçlarının yazdırılması
print()
print("GRU model Score--->",gru_model.evaluate(x_test,y_test,verbose=0))

Model: "GRU_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 400, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 128)            │        62,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 222,337 (868.50 KB)

 Trainable params: 222,337 (868.50 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 445s 943ms/step - accuracy: 0.7194 - loss: 0.5305 - val_accuracy: 0.8594 - val_loss: 0.3682
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 380s 973ms/step - accuracy: 0.8045 - loss: 0.4561 - val_accuracy: 0.8125 - val_loss: 0.5780
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 375s 961ms/step - accuracy: 0.8031 - loss: 0.4504 - val_accuracy: 0.9062 - val_loss: 0.2034
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 372s 955ms/step - accuracy: 0.8959 - loss: 0.2516 - val_accuracy: 0.9688 - val_loss: 0.1694
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 358s 917ms/step - accuracy: 0.9171 - loss: 0.2070 - val_accuracy: 0.9688 - val_loss: 0.1673

GRU model Score---> [0.2838974893093109, 0.8827199935913086]


Bu kod bloğu ile oluşturulan model özetinde, kullanılan katmanların türü, çıkış boyutları ve parametre sayıları görülmektedir. İlk katman olan Embedding, her kelimeyi 32 boyutlu bir vektörle temsil ederek sözcüklerin anlamını daha iyi yansıtacak bir uzaya dönüştürür. Ardından gelen GRU katmanı, 128 nöronla kelime sırasındaki bağımlılıkları işler ve uzun vadeli bağlamı korumada klasik RNN’e göre daha etkili olur. Son olarak Dense katmanı, sigmoid aktivasyonu sayesinde ikili sınıflandırma yapar ve yorumların pozitif ya da negatif olarak etiketlenmesini sağlar. Toplam 222,337 parametrenin tamamı eğitilebilir durumdadır. Bu yapı, hem kelimeler arasındaki bağlamı hem de uzun dizilerdeki bağımlılıkları yakalayarak duygu analizi yapabilecek şekilde tasarlanmıştır.

Eğitim sürecinde modelin performansı giderek artmıştır. İlk epoch’ta doğruluk %71,9 seviyesinde başlamış, beşinci epoch sonunda doğruluk %91,7’ye yükselmiştir. Doğrulama doğruluğu ise %96,8 gibi oldukça yüksek bir seviyeye ulaşmıştır. Kayıp değerleri de her adımda azalmış, bu da modelin veriyi iyi öğrendiğini göstermiştir. Test seti sonuçlarına göre doğruluk %88,2, kayıp ise 0,28 olarak elde edilmiştir. Bu, modelin yeni veriler üzerinde de güçlü bir şekilde genelleme yapabildiğini ortaya koymaktadır.

Sonuçlar, GRU modelinin klasik RNN’e kıyasla daha yüksek bir başarı sağladığını göstermektedir. RNN modeli test doğruluğunda %81,9 seviyesinde kalırken, GRU modeli %88,2 doğruluğa ulaşmıştır. Bunun temel nedeni GRU’nun kapı mekanizmaları sayesinde daha uzun bağımlılıkları öğrenebilmesi ve unutma/hatırlama dengesini daha iyi kurabilmesidir. Dolayısıyla GRU, duygu analizi gibi bağlamın önemli olduğu metin sınıflandırma görevlerinde RNN’e göre daha avantajlı bir yapı sunmaktadır.